In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd
from matplotlib import pyplot as plt
#import dvu
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import viz
import numpy as np
#import torch
import scipy
from matplotlib import pyplot as plt
from sklearn import metrics
import data
from config import *
from tqdm import tqdm
import pickle as pkl
import train_reg
from copy import deepcopy
import config
import models
import pandas as pd
import features
import outcomes
import neural_networks
from sklearn.model_selection import KFold
#from torch import nn, optim
#from torch.nn import functional as F
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.svm import SVR
from collections import defaultdict

# compare all different models across dsets

In [3]:
!ls ../reports/*vps*

../reports/cell_vps_level_res_y_consec_sig.csv
../reports/dataset_vps_level_res_y_consec_sig.csv


In [4]:
suffix = '_y_consec_sig' # '_successful', _successful_dynamin, _successful_full # '' choose which outcome to use

In [5]:
res = pd.read_csv(f"../reports/dataset_vps_level_res{suffix}.csv", index_col=0)

In [6]:
res = pd.read_csv(f"../reports/dataset_vps_level_res{suffix}.csv", index_col=0)
res = res.round(3)
# res = res[[c for c in res.columns if 'balanced' not in c]]
# print(res)
res = res.transpose()
res = res[['gb_basic', 'rf_basic', 'svm_basic', 'svm_dasc', 'lstm']]
num_cells = pkl.load(open("../reports/num_tracks_by_cell.pkl", "rb"))
# res.to_latex()

In [ ]:
res = pd.read_csv(f"../reports/cell_vps_level_res{suffix}.csv", index_col=0)
plot_label = {"accuracy": "accuracy", "f1": "F1 score", "roc_auc": "ROC AUC"}
res = res[[c for c in res.columns if not 'balanced' in c]]

for metric in ['accuracy', 'f1', 'roc_auc']:
    res_metric = res[[c for c in res.columns if metric in c]]
    baseline = 'svm_dasc'
    for baseline in ['svm_dasc', 'gb_basic']:

        plt.figure(dpi=200, figsize=(4, 3))
        plt.grid(zorder=-1.0)
        plt.scatter(res.loc[baseline].values, 
                    res.loc['lstm'].values, 
                    color='#E26595',
                    label='Cells with dynamin marker',
                    alpha=.8)
        
        lowerlim = min(res_metric.loc[baseline].values) - 0.03
        upperlim = max(res_metric.loc['lstm'].values) + 0.03
        plt.xlim((lowerlim, upperlim))
        plt.ylim((lowerlim, upperlim))
        if baseline == 'svm_dasc':
            plt.xlabel(f"DASC {plot_label[metric]}")
        else:
            plt.xlabel(f"Gradient boosting {plot_label[metric]}")
        plt.ylabel(f"LSTM {plot_label[metric]}")
        plt.legend(loc='lower right', fontsize='small', framealpha=1, edgecolor='black')
        plt.plot((lowerlim, upperlim), (lowerlim, upperlim), '--', color='k', alpha=.5)
#         viz.savefig(f'lstm_{baseline}_{metric}')
        plt.show()

In [8]:
res.round(2) #.style.background_gradient()
# res.round(2).style.background_gradient()

,test_accuracy,test_roc_auc,test_r2,test_corr,test_recall,test_f1
gb_basic,0.55,0.72,0.13,0.39,0.81,0.37
gb_dasc,0.37,0.69,-0.25,0.26,0.92,0.33
rf_basic,0.47,0.73,0.06,0.38,0.87,0.36
rf_dasc,0.42,0.62,-0.47,0.18,0.85,0.33
ridge_basic,0.46,0.74,0.12,0.41,0.88,0.36
ridge_dasc,0.36,0.71,-0.10,0.32,0.92,0.33
svm_basic,0.78,0.72,0.05,0.36,0.36,0.36
svm_dasc,0.55,0.71,-0.02,0.32,0.80,0.38
lstm,0.63,0.72,0.13,0.37,0.74,0.40
